In [170]:
import jupyterlab_nodeeditor as jlne
import ipywidgets
from yggdrasil.runner import YggRunner

In [171]:
def clean_bytes(ibytes):
    sbytes = str(ibytes)[1:].strip("'")
    if "\\n" in sbytes:
        clean = sbytes[:-2]
    else:
        clean = sbytes
    return clean

In [200]:
ne = jlne.NodeEditor()
ne

In [202]:
class CBJLNE:
    if len(ne.node_editor.nodes[2].display_element.children) <= 5:
        for i in range(3):
            ne.node_editor.nodes[2].display_element.children += (ipywidgets.Label("Initializing"),)
    
    def __init__(self):
        self.label = ipywidgets.Label()
    
    def display_number(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[3].value = f"Number: {args[0].args[0]}"
        
    def display_string(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[4].value = f"String: {clean_bytes(args[0].args)}"
        
    def display_rotated(self, *args, **kwargs):
        print(args[0].args)
        ne.node_editor.nodes[2].display_element.children[5].value = f"Rotated: {args[0].args}"
    
display = CBJLNE()

In [203]:
runner = YggRunner("yggdrasil_test_models/model_trifecta.yml")

In [204]:
runner.connectiondrivers['model_reader:inputReader']['callbacks'] = [display.display_string]
runner.connectiondrivers['model_rng:outputRNG,model_reader:outputReader_to_model_rotxx:input_rotxx']['callbacks'] = [display.display_number]
runner.connectiondrivers['model_rotxx:output_rotxx']['callbacks'] = [display.display_rotated]

In [205]:
runner.run()

INFO:12496:runner.startDrivers[804]:YggRunner(runner): Starting I/O drivers and models on system None in namespace yggdrasil with rank 0
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\model_rng_test.py
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\model_reader_test.py
X:\anaconda3\envs\yggJLNE\python.exe X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\wip_notebooks\yggdrasil_test_models\ygg_model_rotxx_test_model_rotxx.py
mxlm ebgx 1.

cnbc urwn 2.

INFO:12496:runner.waitModels[871]:YggRunner(runner): model_reader finished running.
INFO:12496:runner.waitModels[877]:YggRunner(runner): model_reader finished exiting.
mxlm ebgx 3.

estd xzopw htww hzcv htes xfns wzyrpc epie qtwpd ld hpww.
End of input from temp_x.
INFO:12496:runner.waitModels[871]:YggRunner(runner): model_

OrderedDict([('init', 1647035935.5911915),
             ('load drivers', 1647035935.6247203),
             ('start drivers', 1647035935.7587488),
             ('run models', 1647035943.3858676),
             ('at exit', 1647035943.3933754)])